In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_openai import ChatOpenAI

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# This is the data structure that will be passed between the nodes.
class GraphState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# --- Define the nodes and router for our graph ---

def call_llm(state: GraphState):
    # This is the main node for the chatbot's conversation turns.
    print("--- Calling LLM ---")
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(state['messages'])
    return {"messages": [response]}

def summarize_conversation(state: GraphState):
    # This node is called when the conversation gets too long.
    print("\n--- SUMMARIZING CONVERSATION ---")
    llm = ChatOpenAI(model="gpt-4o")
    
    # We'll create a summary of the conversation history.
    summary_prompt = "Summarize the following conversation concisely. Include key names, places, and topics discussed:\n\n" + "\n".join(
        [f"{msg.type}: {msg.content}" for msg in state['messages']]
    )
    summary = llm.invoke(summary_prompt).content
    print(f"Generated Summary: {summary}")
    
    # And then replace the long history with this short summary.
    new_messages = [AIMessage(content=f"Conversation summary: {summary}")]
    return {"messages": new_messages}

def should_summarize(state: GraphState):
    # This is the router. It decides which node to go to next.
    print(f"Current message count: {len(state['messages'])}")
    if len(state['messages']) > 4:
        # If the conversation is long, we'll summarize it.
        return "summarize_conversation"
    else:
        # Otherwise, we just end the current turn.
        return "continue_conversation"

# --- Build the graph ---
workflow = StateGraph(GraphState)

workflow.add_node("conversation", call_llm)
workflow.add_node("summarize_conversation", summarize_conversation)

workflow.set_entry_point("conversation")

# This conditional edge calls our router to decide the next step.
workflow.add_conditional_edges(
    "conversation",
    should_summarize,
    {
        "summarize_conversation": "summarize_conversation",
        "continue_conversation": END
    }
)
# After summarizing, we loop back to the conversation node.
workflow.add_edge("summarize_conversation", "conversation")

# --- Set up Memory ---
# The checkpointer saves the state of the graph after each step.
memory = SqliteSaver.from_conn_string(":memory:")

# Compile the graph with memory enabled.
app = workflow.compile(checkpointer=memory)

# --- Run a simulation of a conversation ---
# We use a config with a thread_id to keep the conversation history together.
config = {"configurable": {"thread_id": "my-chat-thread-1"}}

# Turn 1
print("--- Turn 1 ---")
app.invoke({"messages": [HumanMessage(content="Hi! I'm Bob.")]}, config)
print("Turn 1 complete.")

# Turn 2
print("\n--- Turn 2 ---")
app.invoke({"messages": [HumanMessage(content="I'm interested in space exploration.")]}, config)
print("Turn 2 complete.")

# Turn 3
print("\n--- Turn 3 ---")
# This turn will push the message count over the threshold and trigger the summarization.
final_state = app.invoke({"messages": [HumanMessage(content="What's the latest news on the Artemis program?")]}, config)
print("Turn 3 complete.")

print("\n--- Final State of Messages after Summarization ---")
# The final state should now contain the summary.
for msg in final_state['messages']:
    print(f"- {msg.type}: {msg.content}")

--- Turn 1 ---


<frozen importlib._bootstrap>:488: Warning: Numpy built with MINGW-W64 on Windows 64 bits is experimental, and only available for 
testing. You are advised not to use it for production. 

CRASHES ARE TO BE EXPECTED - PLEASE REPORT THEM TO NUMPY DEVELOPERS


: 